In [ ]:
from sentinel_2 import data_manager, Indices
from crops_manager import crops_manager

from shapely.geometry import Point, Polygon
from datetime import datetime
import pandas as pd
import numpy as np
import re
import os

In [ ]:
# read VH data
data = pd.read_csv('./dati.csv', sep=';')

data.loc[:, 'lat'] = data['COORD. GPS'].apply(lambda x: float(x.split(', ')[0]))
data.loc[:, 'long'] = data['COORD. GPS'].apply(lambda x: float(x.split(', ')[1]))

data.loc[:, 'dateref'] = data['DATA EVENTO'].apply(lambda x: [i for i in re.sub('  +', '_', x).split('_') if len(i) > 0])
data = data.explode('dateref')
data['dateref'] = data.dateref.apply(lambda x: datetime.strptime(x, '%d/%m/%Y ; %H:%M'))

In [ ]:
data.head()

In [ ]:
numpy_root_data_dir = '/Volumes/cuda4.deepleey.com/Sentinel2-crops'
# numpy_root_data_dir = '/Volumes/Volume'

tile = 'T32TNQ'

In [ ]:
cm = crops_manager(numpy_root_data_dir, tile)

In [ ]:
evento = data[data.CERTIFICATO == 18625]
evento

### data una coppia di coordinate recupero l'id del crop che le contiene

In [ ]:
cm.getId_by_longlat(evento.long, evento.lat)

In [ ]:
data.loc[:, 'crop'] = data.apply(lambda x: cm.getId_by_longlat(x.long, x.lat), axis=1)
data = data.explode('crop').dropna()

In [ ]:
evento = data[data.CERTIFICATO == 18625].iloc[0]
evento

### dato l'id del crop, restituisce gli shape dei campi individuati

In [ ]:
crops_map = {_id: cm.get_veg_shapes(_id, seg_type='panoptic') for _id in data.crop.unique()}

In [ ]:
data.loc[:, 'panoptic_label'] = data.apply(lambda x: crops_map[x.crop][crops_map[x.crop].contains(Point(x.long, 
                                                                                                        x.lat))].index.values 
                                           if crops_map[x.crop].shape[0] > 0 else None, 
                                           axis=1)
data = data.explode('panoptic_label')

In [ ]:
data.dropna()

### data una coppia di coordinate recupero l'id dei crop con baricentro in un range di distanze (in metri)

In [ ]:
cm.getId_by_distance(evento.long, evento.lat, min_distance=0, max_distance=2000)